In [ ]:
## import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [ ]:
#reading the data set
loan_dataSet = pd.read_csv("D:\loan.csv")

In [ ]:
#Getting the top 5 rows of the data to begin the analysis
loan_dataSet.head()

In [ ]:
loan_dataSet.shape

In [ ]:
# Check for duplicate rows in dataset based on id column

print(loan_dataSet.duplicated(subset=None, keep='first').count())
duplicates=loan_dataSet[loan_dataSet.duplicated(['id'])]
duplicates

# There are no duplicate rows in loan dataset

In [ ]:
# Check for the null values in all the columns in the given data set
loan_dataSet.isnull().sum()

### It is observed that there are a lot of columns with all null values. Hence we can remove/drop those null valued columns as they are not useful for our further analysis

In [ ]:
loan_dataSet.dropna(axis = 1, how = 'all', inplace = True)
loan_dataSet.head()

## Also to be noted that there are several columns which are single valued and they would  not be helpful futher to our analysis. Hence removing them.


In [ ]:
loan_dataSet.drop(['pymnt_plan', "initial_list_status",'collections_12_mths_ex_med','policy_code','acc_now_delinq', 'application_type', 'pub_rec_bankruptcies', 'tax_liens', 'delinq_amnt'], axis = 1, inplace = True)
loan_dataSet.head()

### Now we have 48 columns out of which some correspond to the post approval of loan
- We need to analyze user details and the factors which plays a major role in the loan application defaulting
- We can go ahead and remove the columns that are not useful to our analysis from the data set.
- There are some clmns such as "id", "member_id", "url", "title", "emp_title", "zip_code", "last_credit_pull_d", "addr_state". 
- The above features or columns is not useful to the loan defaulting due to incorrect or not related information. Hence removing them. 
- "desc" has description (text data) which is not useful. Hence removing the column.
- "out_prncp_inv" , "total_pymnt_inv " are also not useful for analysis. Hence removing them. 
- As we have "funded_amnt_inv" , we can remove the funded_amnt column as well as it is not needed

### Columns post approval that can be removed
- delinq_2yrs
- revol_bal
- out_prncp
- total_pymnt
- total_rec_prncp
- total_rec_int
- total_rec_late_fee
- recoveries
- collection_recovery_fee
- last_pymnt_d
- last_pymnt_amnt
- next_pymnt_d
- chargeoff_within_12_mths
- mths_since_last_delinq
- mths_since_last_record

In [ ]:
loan_dataSet.drop(["id", "member_id", "url", "title", "emp_title", "zip_code", "last_credit_pull_d", "addr_state","desc","out_prncp_inv","total_pymnt_inv","funded_amnt", "delinq_2yrs", "revol_bal", "out_prncp", "total_pymnt", "total_rec_prncp", "total_rec_int", "total_rec_late_fee", "recoveries", "collection_recovery_fee", "last_pymnt_d", "last_pymnt_amnt", "next_pymnt_d" , "chargeoff_within_12_mths", "mths_since_last_delinq", "mths_since_last_record"], axis = 1, inplace = True)

In [ ]:
loan_dataSet.shape

In [ ]:
## Actual Columns list which is useful now for the next set of analysis
loan_dataSet.columns

### The main aim of this analysis is to check which user is likely to default based on the fully paid status or if the loan is charged off. Hence there is no use of having current values in the loan_status column


In [ ]:
### Removing the rows with loan status as current
loan_dataSet = loan_dataSet[loan_dataSet.loan_status != "Current"]
### Check the unique values after removing the current loans
loan_dataSet.loan_status.unique()

### Next step is to check for missing values if any in the final list of columns

In [ ]:
(loan_dataSet.isna().sum()/len(loan_dataSet.index))*100

## Handling Missing values
 - columns with missing values are "emp_length", "revol_util".

In [ ]:
### Check the datatype of the columns "emp_length" and "revol_util"
loan_dataSet.info()

In [ ]:
### Check the Mode value of emp_length column

print("Mode value of emp)length coulmn : " + loan_dataSet.emp_length.mode()[0])
loan_dataSet.emp_length.value_counts()

### The mode value has far higher frequency than that of the next most frequent value. 
- Lets format employment length column to have only numbers.

In [ ]:
loan_dataSet.emp_length.fillna(loan_dataSet.emp_length.mode()[0], inplace = True)
loan_dataSet.emp_length.isna().sum()

In [ ]:
loan_dataSet.dropna(axis = 0, subset = ['revol_util'] , inplace = True)
loan_dataSet.revol_util.isna().sum()

## Data Standardization/Correction of data
- "revol_util" column although described as an object column, it has continous values. 
- So we need to standardize the data in this column
- "int_rate" is one such column.
- "emp_length" --> { (< 1 year) is assumed as 0 and 10+ years is assumed as 10 }
- The datatype of "term" is defined as an int, and there onl two values in the whole column and hence it can be considered as categorical column

In [ ]:
loan_dataSet.revol_util = pd.to_numeric(loan_dataSet.revol_util.apply(lambda x : x.split('%')[0]))

In [ ]:
loan_dataSet.int_rate = pd.to_numeric(loan_dataSet.int_rate.apply(lambda x : x.split('%')[0]))

In [ ]:
loan_dataSet.emp_length = pd.to_numeric(loan_dataSet.emp_length.apply(lambda x: 0 if "<" in x else (x.split('+')[0] if "+" in x else x.split()[0])))

In [ ]:
loan_dataSet.head()

## Checking for Outliers in "annual_inc" column

In [ ]:
sns.boxplot(loan_dataSet['annual_inc'])

#### Clearly indicating the presence of outliers.
- Remove the outliers initially
- Check the quantile information
- The values after 95 percentile seems to be disconnected from the general distribution and also there is huge increase in the value for small quantile variation.
- Hence, considering threshold for removing outliers as 0.95

In [ ]:
quantile_info = loan_dataSet.annual_inc.quantile([0.5, 0.75,0.90, 0.95, 0.97,0.98, 0.99])
quantile_info

In [ ]:
per_95_annual_inc = loan_dataSet['annual_inc'].quantile(0.95)
loan_dataSet = loan_dataSet[loan_dataSet.annual_inc <= per_95_annual_inc]

In [ ]:
sns.boxplot(loan_dataSet.annual_inc)

### Now "annual_inc" data looks good and proceeding with next set of  numerical column values
- Analyzing other numerical variables which could possibly have outliers such as the following-
- dti
- loan_amnt
- funded_amnt_inv 

In [ ]:
sns.boxplot(loan_dataSet.dti)

In [ ]:
sns.boxplot(loan_dataSet.loan_amnt)

In [ ]:
loan_dataSet.loan_amnt.quantile([0.75,0.90,0.95,0.97,0.975, 0.98, 0.99, 1.0])

In [ ]:
sns.boxplot(loan_dataSet.funded_amnt_inv)

In [ ]:
loan_dataSet.funded_amnt_inv.quantile([0.5,0.75,0.90,0.95,0.97,0.975, 0.98,0.985, 0.99, 1.0])

- The distribution is continous and there is no need to remove outliers / extreme values for these above columns.

## Visualize Categorical Data Sets
### We already have grade column, extracting only subgrade (int level value) from the sub_grade variable
- We are analyzing and visualizing only the defaulter data. So subsetting the data while plotting only for 'Charged Off' loan_status for below plots


In [ ]:
sns.countplot(x = 'loan_status', data = loan_dataSet)

In [ ]:
loan_dataSet.sub_grade = pd.to_numeric(loan_dataSet.sub_grade.apply(lambda x : x[-1]))
loan_dataSet.sub_grade.head()

In [ ]:
fig, ax = plt.subplots(figsize=(13,8))
sns.set_palette('husl')
sns.countplot(x = 'grade', order = ['A', 'B', 'C', 'D', 'E', 'F', 'G'] , hue = 'sub_grade',data = loan_dataSet[loan_dataSet.loan_status == 'Charged Off'])

In [ ]:
sns.countplot(x = 'grade', data = loan_dataSet[loan_dataSet.loan_status == 'Charged Off'], order = ['A', 'B', 'C', 'D', 'E', 'F', 'G'])

#### Next steps is to analyze home_ownership column

In [ ]:
#checking unique values for home_ownership
loan_dataSet['home_ownership'].unique()


#### There are only 3 records with 'NONE' value in the data. So replacing the value with 'OTHER'

In [ ]:
#replacing 'NONE' with 'OTHERS'
loan_dataSet['home_ownership'].replace(to_replace = ['NONE'],value='OTHER',inplace = True)

In [ ]:
#checking unique values for home_ownership again
loan_dataSet['home_ownership'].unique()

In [ ]:
fig, ax = plt.subplots(figsize = (6,4))
ax.set(yscale = 'log')
sns.countplot(x='home_ownership', data=loan_dataSet[loan_dataSet['loan_status']=='Charged Off'])

####  Analyzing the purpose of loan taken

In [ ]:
fig, ax = plt.subplots(figsize = (12,8))
ax.set(xscale = 'log')
sns.countplot(y ='purpose', data=loan_dataSet[loan_dataSet.loan_status == 'Charged Off'])


#### Creating bins for some numerical variable to make them categorical

In [ ]:
#Lets create additional bins for int_rate,open_acc,revol_util,total_acc based on ranges
loan_dataSet['int_rate_groups'] = pd.cut(loan_dataSet['int_rate'], bins=5,precision =0,labels=['5%-9%','9%-13%','13%-17%','17%-21%','21%-24%'])
loan_dataSet['open_acc_groups'] = pd.cut(loan_dataSet['open_acc'],bins = 5,precision =0,labels=['2-10','10-19','19-27','27-36','36-44'])
loan_dataSet['revol_util_groups'] = pd.cut(loan_dataSet['revol_util'], bins=5,precision =0,labels=['0-20','20-40','40-60','60-80','80-100'])
loan_dataSet['total_acc_groups'] = pd.cut(loan_dataSet['total_acc'], bins=5,precision =0,labels=['2-20','20-37','37-55','55-74','74-90'])
loan_dataSet['annual_inc_groups'] = pd.cut(loan_dataSet['annual_inc'], bins=5,precision =0,labels =['3k-31k','31k-58k','58k-85k','85k-112k','112k-140k'])

In [ ]:
# Viewing new bins created
loan_dataSet.head()

#### Analyzing interest rate w.r.t the interest rate bins created

In [ ]:
fig, ax = plt.subplots(figsize = (12,7))
plt.subplot(221)
sns.countplot(x='int_rate_groups', data=loan_dataSet[loan_dataSet.loan_status == 'Charged Off'])
plt.xlabel('Interest Rate')
plt.subplot(222)
sns.countplot(x='emp_length', data=loan_dataSet[loan_dataSet.loan_status == 'Charged Off'])

#### Similarly analyzing open_acc,revol_util,total_acc,annual_inc

In [ ]:
fig, ax = plt.subplots(figsize = (7,5))
ax.set_yscale('log')
sns.countplot(x='open_acc_groups', data=loan_dataSet[loan_dataSet.loan_status == 'Charged Off'])

In [ ]:
sns.countplot(x='revol_util_groups', data=loan_dataSet[loan_dataSet.loan_status == 'Charged Off'])

In [ ]:
fig, ax = plt.subplots(figsize = (8,6))
ax.set_yscale('log')
sns.countplot(x='total_acc_groups', data=loan_dataSet[loan_dataSet.loan_status == 'Charged Off'])

In [ ]:
fig, ax = plt.subplots(figsize = (8,4))
sns.countplot(x='annual_inc_groups', data=loan_dataSet[loan_dataSet.loan_status == 'Charged Off'])

In [ ]:
sns.countplot(y='term', data=loan_dataSet[loan_dataSet['loan_status']=='Charged Off'])

In [ ]:
sns.countplot(x='verification_status', data=loan_dataSet[loan_dataSet['loan_status']=='Charged Off'])

In [ ]:
fig,ax = plt.subplots(figsize = (8,6))
ax.set_yscale('log')
sns.countplot(x='inq_last_6mths', data=loan_dataSet[loan_dataSet['loan_status']=='Charged Off'])

In [ ]:
fig,ax = plt.subplots(figsize = (7,5))
ax.set_yscale('log')
sns.countplot(x='pub_rec', data=loan_dataSet[loan_dataSet['loan_status']=='Charged Off'])

### Now Lets Analyze the loan issued by month and year


In [ ]:
## Extracting month and year
df_month_year = loan_dataSet['issue_d'].str.partition("-", True)     
loan_dataSet['issue_month']=df_month_year[0]                       
loan_dataSet['issue_year']='20' + df_month_year[2]

In [ ]:
loan_dataSet.head()

In [ ]:
plt.figure(figsize=(15,15))
plt.subplot(221)
sns.countplot(x='issue_month', data=loan_dataSet[loan_dataSet['loan_status']=='Charged Off'])
plt.subplot(222)
sns.countplot(x='issue_year', data=loan_dataSet[loan_dataSet['loan_status']=='Charged Off'])

### Maximum number of loan defaults occured when the it was sanctioned/issued in the month of Dec. Comparison with other years are also shown 

### Analyzing installment,dti, loan_amnt 


In [ ]:
loan_dataSet['installment_groups'] = pd.cut(loan_dataSet['installment'], bins=10,precision =0,labels=['14-145','145-274','274-403','403-531','531-660','660-789','789-918','918-1047','1047-1176','1176-1305'])
loan_dataSet['funded_amnt_inv_group'] = pd.cut(loan_dataSet['funded_amnt_inv'], bins=7,labels=['0-5k','5k-10k','10k-15k','15k-20k','20k-25k','25k-30k','30k-35k'])
loan_dataSet['loan_amnt_groups'] = pd.cut(loan_dataSet['loan_amnt'], bins=7,precision =0,labels=['0-5k','5k-10k','10k-15k','15k-20k','20k-25k','25k-30k','30k-35k'])
loan_dataSet['dti_groups'] = pd.cut(loan_dataSet['dti'], bins=5,precision =0,labels=['0-6','6-12','12-18','18-24','24-30'])

In [ ]:
fig,ax = plt.subplots(figsize = (12,5))
ax.set_yscale('log')
sns.countplot(x='funded_amnt_inv_group', data=loan_dataSet[loan_dataSet['loan_status']=='Charged Off'])

In [ ]:
fig,ax = plt.subplots(figsize = (15,6))
ax.set_yscale('log')
sns.countplot(x='loan_amnt_groups', data=loan_dataSet[loan_dataSet['loan_status']=='Charged Off'])

In [ ]:
sns.countplot(x='dti_groups', data=loan_dataSet[loan_dataSet['loan_status']=='Charged Off'])

In [ ]:
fig,ax = plt.subplots(figsize = (15,6))
ax.set_yscale('log')
sns.countplot(x='installment_groups', data=loan_dataSet[loan_dataSet['loan_status']=='Charged Off'])

# Following are the inferences/observation that we can make based on the plots that we have developed assessing alll the relevant columns

### The last months of an year indicated the high possibility of defaulting.

### The above analysis with respect to the charged off loans for each variable suggests the following. There is a more probability of defaulting when : 

- Applicants having house_ownership as 'RENT'
- Applicants who use the loan to clear other debts
- Applicants who receive interest at the rate of 13-17%
- Applicants who have an income of range 31201 - 58402
- Applicants who have 20-37 open_acc
- Applicants with employement length of 10
- When funded amount by investor is between 5000-10000
- Loan amount is between 5429 - 10357
- Dti is between 12-18
- When monthly installments are between 145-274
- Term of 36 months
- When the loan status is Not verified
- When the purpose is 'debt_consolidation'
- Grade is 'B'


# Now lets see what all observations can we get based on correlation matrix -w.r.t Quantitative Variables
### Bivariate Analysis - Correlation Matrix-Quantitative Variables





In [ ]:
loandata_correlation = loan_dataSet.corr()
sns.clustermap(loandata_correlation,annot=True,figsize=(12, 8),cmap="BrBG")
plt.show()

### Based on the above correlation graph it seems that Loan amount, investor amount, funding amount are strongly correlated.
### Annual income with DTI(Debt-to-income ratio) is negatively correalted.
### Debt income ratio is the percentage of a consumer's monthly gross income that goes toward paying debts. 
### When annual income is low DTI is high & vice versa.
### Positive correlation between annual income and employment years.

## Next steps we can Analyse annual income with other columns as well using Bar plots

#### 1.Annual income vs loan purpose. We can plot a Bar plot in this case

In [ ]:
plt.figure(figsize=(10,10))
sns.barplot(data =loan_dataSet,x='annual_inc', y='purpose', hue ='loan_status',palette="Set2")
plt.show()

### Applicants with higher salary mostly applied loans for "home_improvment", "house", "renewable_energy" and "small_businesses"

#### 2.Annual income vs home ownership

In [ ]:
plt.figure(figsize=(10,10))
sns.barplot(data =loan_dataSet,x='home_ownership', y='annual_inc', hue ='loan_status',palette="husl")
plt.show()

### Annual Income vs Loan amount

In [ ]:
sns.barplot(x = "annual_inc_groups", y = "loan_amnt", hue = 'loan_status', data = loan_dataSet)

### Across all the income groups, the loan_amount is higher for people who defaulted.

#### 3. Annual income vs int_rate

In [ ]:
plt.figure(figsize=(10,10))
sns.barplot(data =loan_dataSet,x='int_rate_groups', y='annual_inc', hue ='loan_status',palette="husl")
plt.show()

## We can Analyse loan_amount with other columns as well

#### 1.Loan Amount vs Interest Rate

In [ ]:
plt.figure(figsize=(10,10))
sns.barplot(data =loan_dataSet,x='loan_amnt_groups', y='int_rate', hue ='loan_status',palette="husl")
plt.show()

#### 2.Loan vs Loan purpose

In [ ]:
plt.figure(figsize=(10,10))
sns.barplot(data =loan_dataSet,x='loan_amnt', y='purpose', hue ='loan_status',palette="husl")
plt.show()

#### 3.Loan vs House Ownership

In [ ]:
plt.figure(figsize=(10,10))
sns.barplot(data =loan_dataSet,x='loan_amnt', y='home_ownership', hue ='loan_status',palette="husl")
plt.show()

#### 4.Loan amount vs month issued and year issued

In [ ]:
plt.figure(figsize=(20,20))
plt.subplot(221)
sns.lineplot(data =loan_dataSet,y='loan_amnt', x='issue_month', hue ='loan_status',palette="husl")
plt.subplot(222)
sns.lineplot(data =loan_dataSet,y='loan_amnt', x='issue_year', hue ='loan_status',palette="husl")


#### 5.Loan amount vs Grade

In [ ]:
plt.figure(figsize=(10,10))
sns.barplot(data =loan_dataSet,x='loan_amnt', y='grade', hue ='loan_status',palette="husl", order=['A','B','C','D','E','F','G'])
plt.show()

#### Loan_amount vs emp_length and verification_status 

In [ ]:
plt.figure(figsize=(20,20))
plt.subplot(221)
sns.barplot(data =loan_dataSet,y='loan_amnt', x='emp_length', hue ='loan_status',palette="husl")
plt.subplot(222)
sns.barplot(data =loan_dataSet,y='loan_amnt', x='verification_status', hue ='loan_status',palette="husl")


### Employees with longer working history got the loan approved for a higher amount. 
- Looking at the verification status data, verified loan applications tend to have higher loan amount. Which might indicate that the firms are first verifying the loans with higher values.

#### grade vs interest rate

In [ ]:
plt.figure(figsize=(8,8))
sns.barplot(data =loan_dataSet,x='int_rate', y='grade', hue ='loan_status',palette="husl", order=['A','B','C','D','E','F','G'])
plt.show()

In [ ]:
# fig,ax = plt.subplots(figsize = (15,6))
plt.tight_layout()
sns.catplot(data =loan_dataSet,y ='int_rate', x ='loan_amnt_groups', hue ='loan_status',palette="husl",kind = 'box')

### The interest rateis very high for the charged off loans than that of fully paid ones based on the derivation from above graph 
- This can be a one of the main reasons for loan defaulting.

In [ ]:
sns.catplot(x = 'term', y = 'loan_amnt', data = loan_dataSet,hue = 'loan_status', kind = 'bar')

### Applicants who applied and defaulted have no significant difference in loan_amounts.
- Which means that applicants applying for long term has applied for more loan. 

# Final Inferences/Observations based on the various retionships between loan_amount and other driving factors for defaulting or a person becoming a loan defaulter


- Users who take loan for 'home improvement' have income of 60k -70k
- Users whose home ownership type is of 'MORTGAGE, have income of 60-70k
- Users who receive interest at the rate of 21%-24%, have an income of 70k-80k
- Users who have taken a loan in the range 30k - 35k and are charged interest rate of 15%-17.5%
- Users who have taken a loan for small business, the loan amount is greater than 14k
- Users whose home ownership is 'MORTGAGE, have loan of 14-16k
- When grade is F then loan amount is between 15k-20k
- When employment length is 10yrs, loan amount is 12k-14k 
- When the loan is verified and loan amount is above 16k
- For grade G, the interest rate is above 20%